## Modeling ##

In [2]:
# pip install faiss_imputer

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    r2_score,
    mean_squared_error, 
    root_mean_squared_error,
    mean_absolute_error, 
    mean_absolute_percentage_error
)
from sklearn.inspection import PartialDependenceDisplay
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from scipy.spatial import KDTree
from faiss_imputer import FaissImputer

from tqdm.notebook import tqdm, trange
from ipywidgets import interactive
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer

This notebook is dedicated to the feature selection and statistical modeling of our trucking data.

In [5]:
df = pd.read_csv('../data/data_clean.csv', low_memory=False)

In [6]:
#df_sample = df.sample(frac=0.3, random_state=42)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057049 entries, 0 to 1057048
Data columns (total 47 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1057049 non-null  int64  
 1   RecordID                   1057049 non-null  int64  
 2   ESS_Id                     1057049 non-null  int64  
 3   EventTimeStamp             1057049 non-null  object 
 4   eventDescription           1006152 non-null  object 
 5   ecuSoftwareVersion         792713 non-null   object 
 6   ecuSerialNumber            751116 non-null   object 
 7   ecuModel                   1001026 non-null  object 
 8   ecuMake                    1001026 non-null  object 
 9   ecuSource                  1057049 non-null  int64  
 10  spn                        1057049 non-null  int64  
 11  fmi                        1057049 non-null  int64  
 12  active                     1057049 non-null  bool   
 13  activeTransi

In [8]:
df = df.drop('Unnamed: 0', axis=1)

In [9]:
object_columns_to_change_to_category = [        
    'eventDescription',  
    'ecuSoftwareVersion',          
    'ecuSerialNumber',         
    'ecuModel',           
    'ecuMake',                  
    'EquipmentID',                                 
    'LampStatus',              
    'ServiceDistance',
    'next_derate_timestamp',
    'time_until_detate'] 

In [10]:
for column in object_columns_to_change_to_category:
    df[column] = df[column].astype('category')

In [11]:
df['EventTimeStamp'] = pd.to_datetime(df['EventTimeStamp'])
df['LocationTimeStamp'] = pd.to_datetime(df['LocationTimeStamp'])

In [12]:
int_columns_to_categorical = ['RecordID',
'ESS_Id',                    
'ecuSource',                    
'spn',                       
'fmi',  
'active',       
'MCTNumber',                  
'FaultId']

In [13]:
for column in int_columns_to_categorical:
    df[column] = df[column].astype('category')

In [14]:
float64_cols = df.select_dtypes(include=['float64']).columns
df[float64_cols] = df[float64_cols].astype('float32')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057049 entries, 0 to 1057048
Data columns (total 46 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   RecordID                   1057049 non-null  category      
 1   ESS_Id                     1057049 non-null  category      
 2   EventTimeStamp             1057049 non-null  datetime64[ns]
 3   eventDescription           1006152 non-null  category      
 4   ecuSoftwareVersion         792713 non-null   category      
 5   ecuSerialNumber            751116 non-null   category      
 6   ecuModel                   1001026 non-null  category      
 7   ecuMake                    1001026 non-null  category      
 8   ecuSource                  1057049 non-null  category      
 9   spn                        1057049 non-null  category      
 10  fmi                        1057049 non-null  category      
 11  active                     1057049 no

Scaling and encoding features for modeling

In [17]:
X = df.drop(['target',
            'LocationTimeStamp',
            'EventTimeStamp',
            'eventDescription',
            'ESS_Id',
            'ecuModel',
            'ecuMake',
            'ecuSource',
            'ecuSoftwareVersion',
            'ecuSerialNumber',
            'MCTNumber',
            'ServiceDistance',
            'Latitude',
            'Longitude',
            'RecordID',
            'next_derate_timestamp',
            'time_until_detate',
            'FaultId'], axis=1)

y = df['target'].values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=27, stratify=y)

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 739934 entries, 1035968 to 850462
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   spn                        739934 non-null  category
 1   fmi                        739934 non-null  category
 2   active                     739934 non-null  category
 3   activeTransitionCount      739934 non-null  int64   
 4   EquipmentID                739934 non-null  category
 5   AcceleratorPedal           336870 non-null  float32 
 6   BarometricPressure         371934 non-null  float32 
 7   CruiseControlActive        739934 non-null  bool    
 8   CruiseControlSetSpeed      365583 non-null  float32 
 9   DistanceLtd                371835 non-null  float32 
 10  EngineCoolantTemperature   371915 non-null  float32 
 11  EngineLoad                 371599 non-null  float32 
 12  EngineOilPressure          372018 non-null  float32 
 13  EngineOilTemp

In [20]:
categorical_features = ['spn', 'fmi', 'EquipmentID', 'LampStatus'] 
bool_features = ['CruiseControlActive', 'ParkingBrake', 'IgnStatus', 'active']
numeric_features = [x for x in X_train.columns if x not in categorical_features + bool_features] 

numeric_pipe = Pipeline(
    steps=[
        ('scale', StandardScaler()),
        ('faiss_imputer', FaissImputer(n_neighbors=3, strategy = 'mean')),
    ]
)

categorical_pipe = Pipeline(
    steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
        ('KNN_imputer', SimpleImputer(strategy='most_frequent')),
    ]
)

bool_pipe = Pipeline(
    steps=[
        ('bool_impute', SimpleImputer(strategy='most_frequent'))
    ]
)

ct = ColumnTransformer(
    transformers=[
        ('numeric', numeric_pipe, numeric_features),
        ('categorical', categorical_pipe, categorical_features),
        ('bool', bool_pipe, bool_features)
    ]
)

pipe = Pipeline(
    steps=[
        ('transformer', ct),
        ('lightgbm', LGBMClassifier(
        ))
    ]
).fit(X_train, y_train)

C:\Users\billy\anaconda3\Lib\site-packages\faiss_imputer\faiss_imputer.py:99: RuntimeWarning: Mean of empty slice
  column_agg = np.nanmean(selected_values, axis=0)


[LightGBM] [Info] Number of positive: 1763, number of negative: 738171
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.126267 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6531
[LightGBM] [Info] Number of data points in the train set: 739934, number of used features: 1270
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002383 -> initscore=-6.037159
[LightGBM] [Info] Start training from score -6.037159


In [21]:
# - there rae 1763 positive samples and 738,171 negative samples. This suggests dataset is highly imbalanced, which may require adjustments like class weighting or resampling.
# - LightGBM uses histogram-based learning instead of splitting values directly. This means the dataset has 6531 unique bin partitions, which can improve training efficiency.
# - there are 739,934 rows being used for training. the model is leveraging 1270 features in total.
# - Since it's a binary classification task, LightGBM calculates an initial score (-6.037159). This score is derived from the prior probability (pavg=0.002383), meaning only 0.238% of samples are positive. The low prior probability affects LightGBM's initial weight adjustments to balance the learning process.

# Next steps:
# Given the severe class imbalance, I might: ✔ Use scale_pos_weight in LightGBM to balance classes ✔ Apply oversampling techniques (SMOTE, ADASYN) if necessary ✔ Tune the learning rate for better convergence

In [22]:
y_pred = pipe.predict(X_test)

C:\Users\billy\anaconda3\Lib\site-packages\faiss_imputer\faiss_imputer.py:99: RuntimeWarning: Mean of empty slice
  column_agg = np.nanmean(selected_values, axis=0)


In [23]:
print(classification_report(y_test, y_pred, zero_division = 0))
#feature_importances = pd.Series(pipe.feature_importances_, index=X_train.columns)
#feature_importances.sort_values(ascending=False)
#print(feature_importances)

              precision    recall  f1-score   support

       False       1.00      1.00      1.00    316360
        True       0.29      0.24      0.26       755

    accuracy                           1.00    317115
   macro avg       0.64      0.62      0.63    317115
weighted avg       1.00      1.00      1.00    317115



In [24]:
df[df.isin(['99,6']).any(axis=1)]

C:\Users\billy\AppData\Local\Temp\ipykernel_13004\3329684240.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df[df.isin(['99,6']).any(axis=1)]


,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,next_derate_timestamp,time_until_detate,target


In [25]:
df.loc[df['EquipmentID'] == 'R1762']['EquipmentID'].astype(float)

ValueError: Cannot cast object dtype to float64